In [1]:
import sys
from pathlib import Path

ROOT = Path(".").resolve()   # notebooks/ 동일 폴더
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from kis_nine import *
from kis_ten import *

## Step 11-0. 일봉 시계열 & 리스크 요약 저장 폴더

퀀트/리스크 분석용 데이터는 두 단계로 나눠 저장합니다.

1. "원시 일봉 시계열" (ETF, 벤치마크 모두)
   - 경로: `data/daily_prices/`
   - 예: `data/daily_prices/069500_daily_2023-01-01_2024-01-31.parquet`

2. "리스크/퀀트 지표 요약" (ETF 단위, 벤치마크까지 포함)
   - 경로: `data/etf_risk_summary/`
   - 예: `data/etf_risk_summary/069500_risk_2024-01-31.parquet`

이후 강의에서는:
- 이 파일들을 읽어와서
- 여러 ETF/기간 간 리스크 지표를 비교/시각화하는 파트를 진행합니다.

In [2]:
DAILY_PRICE_DIR = DATA_DIR / "daily_prices"
RISK_SUMMARY_DIR = DATA_DIR / "etf_risk_summary"

DAILY_PRICE_DIR.mkdir(parents=True, exist_ok=True)
RISK_SUMMARY_DIR.mkdir(parents=True, exist_ok=True)

print("DAILY_PRICE_DIR :", DAILY_PRICE_DIR)
print("RISK_SUMMARY_DIR:", RISK_SUMMARY_DIR)

DAILY_PRICE_DIR : /Users/ulift/workspace/py-etf-mango/script2/data/daily_prices
RISK_SUMMARY_DIR: /Users/ulift/workspace/py-etf-mango/script2/data/etf_risk_summary


## Step 11-1. 국내주식 기간별 시세(일봉) 1회 호출 유틸

- 1회 호출은 최대 100개 일봉까지만 반환됩니다.
- 여기서는 "한 번 요청"만 담당하는 함수와,
- 이후에 1년/3년 데이터를 위해 여러 번 호출을 이어붙이는 함수를 분리합니다.

In [3]:
from datetime import datetime
from typing import Optional, Tuple
import pandas as pd

def fetch_daily_price_once(
    code: str,
    start_date: str,
    end_date: str,
    *,
    period_div: str = "D",
    org_adj_prc: str = "0",
) -> pd.DataFrame:
    """
    국내주식 기간별시세 API를 '한 번' 호출해서
    최대 100개 일봉을 DataFrame으로 반환.

    인자:
    - code: 종목코드(6자리, ETF 포함)
    - start_date: 조회 시작일 (YYYYMMDD)
    - end_date: 조회 종료일 (YYYYMMDD)
    - period_div: 'D' (일봉)
    - org_adj_prc: '0' (수정주가)

    반환:
    - DataFrame (일자별 시세), 최신일자가 "마지막 행"이 되도록 정렬.
    """
    path = "/uapi/domestic-stock/v1/quotations/inquire-daily-itemchartprice"
    tr_id = "FHKST03010100"

    params = {
        "FID_COND_MRKT_DIV_CODE": "J",          # 코스피/코스닥
        "FID_INPUT_ISCD": str(code).zfill(6),   # 종목코드
        "FID_INPUT_DATE_1": start_date,         # 조회 시작일 (YYYYMMDD)
        "FID_INPUT_DATE_2": end_date,           # 조회 종료일 (YYYYMMDD)
        "FID_PERIOD_DIV_CODE": period_div,      # D: 일
        "FID_ORG_ADJ_PRC": org_adj_prc,         # 0: 수정주가
    }

    data = call_kis_get(path, tr_id, params)
    rows = data.get("output2") or []

    if not rows:
        print(f"⚠️ 일봉 데이터가 비어 있습니다. code={code}, {start_date}~{end_date}")
        return pd.DataFrame()

    df = pd.DataFrame(rows)

    # 컬럼 정리
    df["code"] = str(code).zfill(6)
    # 날짜: YYYYMMDD → datetime, YYYY-MM-DD 문자열도 같이 만들어 둠
    df["date"] = pd.to_datetime(df["stck_bsop_date"], format="%Y%m%d")
    df["date_str"] = df["date"].dt.strftime("%Y-%m-%d")

    # 숫자컬럼 변환
    for col in ["stck_clpr", "stck_oprc", "stck_hgpr", "stck_lwpr", "acml_vol"]:
        if col in df.columns:
            df[col + "_float"] = df[col].apply(to_float)

    # 정렬: 오래된 날짜 → 최신 날짜
    df = df.sort_values("date").reset_index(drop=True)

    return df

## Step 11-2. 1년 이상 데이터(>100일)를 위한 구간 반복 호출

- KIS API는 한 번에 최대 100개 레코드만 반환하므로,
- `chunk_size`(예: 90일) 단위로 기간을 쪼개서 여러 번 호출 후,
- 하나의 DataFrame으로 이어붙입니다.

In [4]:
from datetime import timedelta

def fetch_daily_price_range(
    code: str,
    start_date: str,
    end_date: str,
    *,
    chunk_size: int = 90,
) -> pd.DataFrame:
    """
    국내주식 기간별시세 API를 여러 번 호출하여,
    긴 기간(1~3년)의 일봉 데이터를 모두 수집하고 이어붙이는 함수.

    인자:
    - code: 종목코드(6자리)
    - start_date: 조회 시작일 (YYYYMMDD)
    - end_date: 조회 종료일 (YYYYMMDD)
    - chunk_size: 한 번에 호출할 최대 일수 (90~100 권장)

    반환:
    - 전체 기간에 대한 일봉 시세 DataFrame (날짜 오름차순 정렬, 중복 제거).
    """
    code = str(code).zfill(6)

    dt_start = datetime.strptime(start_date, "%Y%m%d").date()
    dt_end = datetime.strptime(end_date, "%Y%m%d").date()

    if dt_start > dt_end:
        raise ValueError(f"start_date({start_date})가 end_date({end_date})보다 큽니다.")

    frames: list[pd.DataFrame] = []
    cur_start = dt_start

    while cur_start <= dt_end:
        cur_end = min(cur_start + timedelta(days=chunk_size - 1), dt_end)

        s = cur_start.strftime("%Y%m%d")
        e = cur_end.strftime("%Y%m%d")

        print(f"[INFO] fetch {code} {s} ~ {e}")
        df_chunk = fetch_daily_price_once(code, s, e)
        if not df_chunk.empty:
            frames.append(df_chunk)

        cur_start = cur_end + timedelta(days=1)

    if not frames:
        raise RuntimeError(f"일봉 데이터를 가져오지 못했습니다. code={code}, {start_date}~{end_date}")

    df_all = pd.concat(frames, ignore_index=True)

    # 중복 일자 제거 (있을 수 있으므로)
    df_all = df_all.sort_values("date").drop_duplicates(subset=["code", "date"]).reset_index(drop=True)

    return df_all

## Step 11-3. 단일 자산(ETF) 기준 리스크 지표 계산

필요 재료:
- `close` 가격 시계열 (여기서는 `stck_clpr_float`)

계산 항목:
- daily_ret: 일간 수익률
- 누적수익률: 구간별로 나중에 편하게 계산할 수 있도록 함수 분리
- 변동성(연환산): std(daily_ret) * sqrt(252)
- 최대낙폭(MDD): (price / price.cummax() - 1).min()
- Sharpe, Sortino: Pandas 연산으로 구현

In [5]:
import numpy as np

def enrich_with_returns(df_price: pd.DataFrame) -> pd.DataFrame:
    """
    일봉 시세 DataFrame에 일간 수익률 컬럼을 추가.

    요구 컬럼:
    - stck_clpr_float: 종가 (숫자)
    - date: 일자(datetime)
    """
    df = df_price.copy()

    if "stck_clpr_float" not in df.columns:
        raise ValueError("stck_clpr_float 컬럼이 필요합니다. fetch_daily_price_* 함수가 올바르게 호출되었는지 확인하세요.")

    df = df.sort_values("date").reset_index(drop=True)
    df["daily_ret"] = df["stck_clpr_float"].pct_change()

    return df


def calc_period_return(df_ret: pd.DataFrame, periods: int) -> Optional[float]:
    """
    최근 N 거래일(periods)에 대한 누적 수익률 계산.
    - df_ret는 daily_ret 컬럼을 포함해야 하며 날짜 오름차순 가정.
    - periods가 데이터 길이보다 길면 None 반환.
    """
    df = df_ret.dropna(subset=["daily_ret"]).copy()
    if len(df) < periods:
        return None

    last_slice = df.iloc[-periods:]
    cum = (1.0 + last_slice["daily_ret"]).prod() - 1.0
    return float(cum)


def calc_volatility_annual(df_ret: pd.DataFrame) -> float:
    """
    일간 수익률 기반 연환산 변동성(표준편차) 계산.
    """
    daily = df_ret["daily_ret"].dropna()
    if len(daily) < 2:
        return float("nan")
    return float(daily.std(ddof=1) * np.sqrt(252))


def calc_mdd(df_price: pd.DataFrame) -> float:
    """
    최대낙폭(MDD) 계산.
    - price / price.cummax() - 1 의 최소값.
    """
    price = df_price["stck_clpr_float"].dropna()
    if price.empty:
        return float("nan")
    cummax = price.cummax()
    dd = price / cummax - 1.0
    return float(dd.min())


def calc_sharpe_sortino(
    df_ret: pd.DataFrame,
    *,
    risk_free_rate: float = 0.03,  # 연 3% 기본
) -> Tuple[float, float]:
    """
    Sharpe, Sortino 비율 계산.
    - risk_free_rate: 연 단위 (예: 0.03 = 3%)
    """
    daily = df_ret["daily_ret"].dropna()
    if len(daily) < 2:
        return float("nan"), float("nan")

    mean_daily = daily.mean()
    std_daily = daily.std(ddof=1)
    downside = daily[daily < 0]
    downside_std = downside.std(ddof=1) if len(downside) > 0 else float("nan")

    # 연환산 기대수익률
    ann_ret = mean_daily * 252
    # 연 무위험수익률
    rf_ann = risk_free_rate

    # Sharpe
    if std_daily > 0:
        sharpe = (ann_ret - rf_ann) / (std_daily * np.sqrt(252))
    else:
        sharpe = float("nan")

    # Sortino
    if isinstance(downside_std, float) and downside_std > 0:
        sortino = (ann_ret - rf_ann) / (downside_std * np.sqrt(252))
    else:
        sortino = float("nan")

    return float(sharpe), float(sortino)

## Step 11-4. ETF vs 시장지수 베타/알파 계산

입력:
- df_etf: ETF 일봉 + daily_ret 포함
- df_mkt: 시장지수 일봉 + daily_ret 포함

절차:
1. 날짜 기준 inner join
2. ETF, Market 일간 수익률 컬럼 생성
3. Beta, Alpha 계산

In [6]:
def calc_beta_alpha(
    df_etf: pd.DataFrame,
    df_mkt: pd.DataFrame,
    *,
    risk_free_rate: float = 0.03,
) -> Tuple[float, float]:
    """
    ETF vs Market 일간 수익률을 기반으로 Beta, Alpha 계산.

    - df_etf, df_mkt 모두:
      - 'date' 컬럼: datetime
      - 'daily_ret' 컬럼: 일간 수익률
    """
    left = df_etf[["date", "daily_ret"]].rename(columns={"daily_ret": "ret_etf"})
    right = df_mkt[["date", "daily_ret"]].rename(columns={"daily_ret": "ret_mkt"})

    merged = pd.merge(left, right, on="date", how="inner").dropna()
    if len(merged) < 10:
        print("⚠️ Beta/Alpha 계산을 위한 공통 거래일이 너무 적습니다.")
        return float("nan"), float("nan")

    ret_etf = merged["ret_etf"]
    ret_mkt = merged["ret_mkt"]

    var_mkt = ret_mkt.var(ddof=1)
    if var_mkt <= 0:
        return float("nan"), float("nan")

    cov = np.cov(ret_etf, ret_mkt, ddof=1)[0, 1]
    beta = cov / var_mkt

    # 연환산 기대수익률
    mean_etf_daily = ret_etf.mean()
    mean_mkt_daily = ret_mkt.mean()

    ann_ret_etf = mean_etf_daily * 252
    ann_ret_mkt = mean_mkt_daily * 252
    rf_ann = risk_free_rate

    # CAPM 기대수익률
    capm_expected = rf_ann + beta * (ann_ret_mkt - rf_ann)
    alpha = ann_ret_etf - capm_expected

    return float(beta), float(alpha)

## Step 11-5. ETF 퀀트/리스크 지표 전체 파이프라인 + 저장

하나의 엔트리 포인트:

```python
run_etf_risk_job(
    etf_code="069500",
    benchmark_code="000020",        # 예시용, 실제 지수 코드로 교체
    start_date="20230101",
    end_date="20241231",
)
```

- 일봉 데이터(ETF + BM)를 parquet로 저장
- 리스크 요약 지표를 parquet로 저장
- 계산된 DataFrame과 요약 dict를 반환

In [ ]:
def run_etf_risk_job(
    etf_code: str,
    benchmark_code: str,
    start_date: str,
    end_date: str,
    *,
    risk_free_rate: float = 0.03,
    chunk_size: int = 90,
) -> dict[str, Any]:
    """
    ETF + 벤치마크에 대해:
    1) 기간별 일봉 데이터 수집 (100개 제한 처리)
    2) 일간 수익률/리스크 지표 계산
    3) Beta/Alpha 계산
    4) 결과를 parquet로 저장하고 dict로 반환.

    저장 파일 예:
    - data/daily_prices/069500_daily_2023-01-01_2024-01-31.parquet
    - data/daily_prices/000020_daily_2023-01-01_2024-01-31.parquet
    - data/etf_risk_summary/069500_risk_2024-01-31.parquet
    """
    etf_code = str(etf_code).zfill(6)
    benchmark_code = str(benchmark_code).zfill(6)

    # 1) 일봉 시계열 수집 (ETF + BM)
    df_etf_raw = fetch_daily_price_range(etf_code, start_date, end_date, chunk_size=chunk_size)
    df_bm_raw = fetch_daily_price_range(benchmark_code, start_date, end_date, chunk_size=chunk_size)

    # 2) 일간 수익률 컬럼 추가
    df_etf = enrich_with_returns(df_etf_raw)
    df_bm = enrich_with_returns(df_bm_raw)

    # 3) ETF 단일 리스크 지표
    vol_ann = calc_volatility_annual(df_etf)
    mdd = calc_mdd(df_etf)
    sharpe, sortino = calc_sharpe_sortino(df_etf, risk_free_rate=risk_free_rate)

    # 4) Beta / Alpha
    beta, alpha = calc_beta_alpha(df_etf, df_bm, risk_free_rate=risk_free_rate)

    # 5) 기간별 누적수익률 (예: 1M/3M/6M/1Y)
    #   - 거래일 기준으로, 대략 21/63/126/252일
    period_map = {
        "1M": 21,
        "3M": 63,
        "6M": 126,
        "1Y": 252,
    }
    period_returns: dict[str, Optional[float]] = {}
    for label, days in period_map.items():
        period_returns[label] = calc_period_return(df_etf, days)

    # 6) 요약 테이블 생성
    if not df_etf.empty:
        as_of_date = df_etf["date"].max().strftime("%Y-%m-%d")
    else:
        as_of_date = datetime.strptime(end_date, "%Y%m%d").date().strftime("%Y-%m-%d")

    summary_row = {
        "as_of_date": as_of_date,
        "etf_code": etf_code,
        "benchmark_code": benchmark_code,
        "risk_free_rate": risk_free_rate,
        "vol_annual": vol_ann,
        "mdd": mdd,
        "sharpe": sharpe,
        "sortino": sortino,
        "beta": beta,
        "alpha": alpha,
    }
    # 기간별 수익률 병합
    for label, val in period_returns.items():
        summary_row[f"return_{label}"] = val

    df_summary = pd.DataFrame([summary_row])

    # 7) 파일 저장
    # 일봉 파일 이름용 표시 날짜 범위
    s_disp = datetime.strptime(start_date, "%Y%m%d").strftime("%Y-%m-%d")
    e_disp = datetime.strptime(end_date, "%Y%m%d").strftime("%Y-%m-%d")

    etf_price_path = DAILY_PRICE_DIR / f"{etf_code}_daily_{s_disp}_{e_disp}.csv"
    bm_price_path = DAILY_PRICE_DIR / f"{benchmark_code}_daily_{s_disp}_{e_disp}.csv"
    risk_path = RISK_SUMMARY_DIR / f"{etf_code}_risk_{as_of_date}.csv"

    df_etf.to_csv(etf_price_path, index=False)
    df_bm.to_csv(bm_price_path, index=False)
    df_summary.to_csv(risk_path, index=False)

    print(f"[SAVE] ETF daily → {etf_price_path}")
    print(f"[SAVE] BM  daily → {bm_price_path}")
    print(f"[SAVE] risk summary → {risk_path}")

    return {
        "etf_daily": df_etf,
        "bm_daily": df_bm,
        "risk_summary": df_summary,
        "period_returns": period_returns,
    }

## Step 11-6. 예시 실행

- ETF: KODEX 200 (`069500`)
- 벤치마크: (예시) `069500` 동일 코드 또는 실제 지수 코드로 교체
- 기간: 최근 2년 (예: 2023-01-01 ~ 2024-12-31)

주의:
- 실제로는 "지수 전용 코드"를 따로 정해서 사용하는 게 더 정교한 방법입니다.
- 여기서는 강의 흐름상 하나의 코드만 바꿔서 테스트 가능하게 해 둡니다.


In [8]:
example_etf = "069500"
example_benchmark = "069500"  # 실제 지수 코드로 교체하는 것을 권장

start_date = "20230101"
end_date = "20241231"

ensure_kis_token()

risk_result = run_etf_risk_job(
    etf_code=example_etf,
    benchmark_code=example_benchmark,
    start_date=start_date,
    end_date=end_date,
    risk_free_rate=0.03,
    chunk_size=90,
)

print("\n[ETF daily] head:")
print(risk_result["etf_daily"].head())

print("\n[risk_summary]:")
print(risk_result["risk_summary"])

[INFO] fetch 069500 20230101 ~ 20230331
[INFO] fetch 069500 20230401 ~ 20230629
[INFO] fetch 069500 20230630 ~ 20230927
[INFO] fetch 069500 20230928 ~ 20231226
[INFO] fetch 069500 20231227 ~ 20240325
[INFO] fetch 069500 20240326 ~ 20240623
[INFO] fetch 069500 20240624 ~ 20240921
[INFO] fetch 069500 20240922 ~ 20241220
[INFO] fetch 069500 20241221 ~ 20241231
[INFO] fetch 069500 20230101 ~ 20230331
[INFO] fetch 069500 20230401 ~ 20230629
[INFO] fetch 069500 20230630 ~ 20230927
[INFO] fetch 069500 20230928 ~ 20231226
[INFO] fetch 069500 20231227 ~ 20240325
[INFO] fetch 069500 20240326 ~ 20240623
[INFO] fetch 069500 20240624 ~ 20240921
[INFO] fetch 069500 20240922 ~ 20241220
[INFO] fetch 069500 20241221 ~ 20241231
[SAVE] ETF daily → /Users/ulift/workspace/py-etf-mango/script2/data/daily_prices/069500_daily_2023-01-01_2024-12-31.parquet
[SAVE] BM  daily → /Users/ulift/workspace/py-etf-mango/script2/data/daily_prices/069500_daily_2023-01-01_2024-12-31.parquet
[SAVE] risk summary → /Users/uli